In [1]:
#loading packages
import os
import glob
import time # adding sleep time between each calls
import random  # random number generator for time

import requests # sending HTTP request
import re
from bs4 import BeautifulSoup

import pandas as pd

#conda install <package name>
#pip install <package name>

In [2]:
#check currect directory
path = os.getcwd()
print ("The current working directory is %s" % path)

The current working directory is /Users/hksahe/Documents/TampereBNB/Jupyter


In [3]:
#creating a directory
newpath = path+'/data/accom'
if not os.path.exists(newpath):
     os.makedirs(newpath)
     print("newpath")

In [4]:
#scraping list of accommodations
for listing_index in range(0, 1080):
     print("getting the" + listing_index + "url:")
     url = 'https://joda-tuni.azurewebsites.net/accom/M20%1d23' % listing_index
     print(url)
     page = requests.get(url)
     fname = 'data/accom/M20%1d23.html' % listing_index
     print(fname)
     with open(fname, 'wb') as f:
         f.write(page.content)
     time.sleep(1 + random.random() * 2)

0
http://127.0.0.1:5000/accom/M20023
data/accom/M20023.html
1
http://127.0.0.1:5000/accom/M20123
data/accom/M20123.html
2
http://127.0.0.1:5000/accom/M20223
data/accom/M20223.html
3
http://127.0.0.1:5000/accom/M20323
data/accom/M20323.html
4
http://127.0.0.1:5000/accom/M20423
data/accom/M20423.html
5
http://127.0.0.1:5000/accom/M20523
data/accom/M20523.html
6
http://127.0.0.1:5000/accom/M20623
data/accom/M20623.html
7
http://127.0.0.1:5000/accom/M20723
data/accom/M20723.html
8
http://127.0.0.1:5000/accom/M20823
data/accom/M20823.html
9
http://127.0.0.1:5000/accom/M20923
data/accom/M20923.html
10
http://127.0.0.1:5000/accom/M201023
data/accom/M201023.html
11
http://127.0.0.1:5000/accom/M201123
data/accom/M201123.html
12
http://127.0.0.1:5000/accom/M201223
data/accom/M201223.html
13
http://127.0.0.1:5000/accom/M201323
data/accom/M201323.html
14
http://127.0.0.1:5000/accom/M201423
data/accom/M201423.html
15
http://127.0.0.1:5000/accom/M201523
data/accom/M201523.html
16
http://127.0.0.1:50

KeyboardInterrupt: 

In [5]:
#reading files
indices = glob.glob('data/accom/*.html')
len(indices)

1080

In [6]:
#loading data from local
local_files = list()
for fname in glob.glob('data/accom/*.html'):
    local_files.append(fname.split('/')[-1])

In [7]:
#speficy keys/columns
column = ["Region", "Price", "Type", "Size", "Floor", "Construction", "Condition", "Elevator", "Longitude", "Latitude"]

In [8]:
#creating dictionary of lists with pre-specified columns/keys
accom = dict()
for col in column:
    accom[col] = list()

In [9]:
counting=0
for fname in indices:
    with open(fname) as f:
        content = f.read()
    soup = BeautifulSoup(content, 'html.parser')
    page_content = soup.find('div', {'class': 'container'})
    #getting the price
    price = page_content.select_one('.basic-info .profile_price').text
    price = price.split(" ")[0].strip()
    accom["Price"].append(price)
    #getting all info on space
    space_info = page_content.find('div', class_='about_space_basic')
    #getting the region
    region = space_info.select_one('.about_space_region p').text
    region = region.split(" ")[-1].replace(".", "")
    accom["Region"].append(region)
    #getting the apartment type
    apartment_type = space_info.select_one('.about_space_type p').text
    accom["Type"].append(apartment_type)
    #getting the apartment size
    apartment_type = space_info.select_one('.about_space_area p').text
    apartment_type = apartment_type.split(" ")[0]
    accom["Size"].append(apartment_type)
    #getting the apartment floor
    floor = space_info.select_one('.about_space_floor p').text
    floor = floor.split(" ")[4]
    accom["Floor"].append(floor)
    #getting the apartment construction year
    year = space_info.select_one('.about_space_year p').text
    year = [int(y) for y in year.split() if y.isdigit()][0]
    accom["Construction"].append(year)
    #getting the apartment status
    status = space_info.find('div', class_='Guest about_space_condition').p.text
    status = status.split()[-2]
    accom["Condition"].append(status)
    #getting the availability of the elevator
    table = soup.find("table")
    elevator = table.select("tr td")[0].text
    counting+=1
    if "does have" in elevator:
        accom["Elevator"].append("Yes")
    else:
        accom["Elevator"].append("No")
    #getting the longitude
    location = soup.find("div", class_="map").p.text
    location = [l for l in location.split()]
    location = list(filter(lambda x: re.match("\d+\.\d+", x), location))
    accom["Longitude"].append(location[0])
    accom["Latitude"].append(location[1][:len(location[1])-1])

In [10]:
#checking the lenght of the key values
print(len(accom["Region"]))
print(len(accom["Price"]))
print(len(accom["Type"]))
print(len(accom["Size"]))
print(len(accom["Floor"]))
print(len(accom["Construction"]))
print(len(accom["Condition"]))
print(len(accom["Elevator"]))
print(len(accom["Longitude"]))
print(len(accom["Latitude"]))

1080
1080
1080
1080
1080
1080
1080
1080
1080
1080


In [11]:
#loading pickle file and getting the first five row
df = pd.DataFrame.from_dict(accom)
df.head()

,Region,Price,Type,Size,Floor,Construction,Condition,Elevator,Longitude,Latitude
0,Tesoma,€180,Three rooms,74.0,7,1966,unknown,Yes,23.61033840613356,61.51083191092233
1,Takahuhti,€243,Two rooms,57.5,4,1972,good,Yes,23.860745947641902,61.49763715174763
2,Hervanta,€177,Studio apartment,27.0,0,2019,good,Yes,23.84882877070141,61.45446021008072
3,Niemenranta,€153,Studio apartment,24.5,1,2017,good,No,23.690050674048194,61.51717521456669
4,Tammela,€363,Three rooms,73.0,4,1971,good,Yes,23.77786391016316,61.50136263215722


In [12]:
#loading pickle file and getting the last five row
df.tail()

,Region,Price,Type,Size,Floor,Construction,Condition,Elevator,Longitude,Latitude
1075,Rautaharkko,€168,Two rooms,42.0,0,1955,unknown,No,23.768104537978544,61.472621213004935
1076,Vuores,€399,Four rooms or more,95.0,-1,2012,good,No,23.808197301985487,61.43451271139134
1077,Hämeenpuisto,€222,Studio apartment,28.0,5,1908,good,No,23.75151211981315,61.49771017598682
1078,Hakametsä,€351,Three rooms,63.0,5,2012,good,Yes,23.829752825287773,61.48832464659127
1079,Ikuri,€366,Four rooms or more,141.0,0,1974,unknown,No,23.59902615472741,61.51101167244906


In [13]:
#saving to pickle file
df.to_pickle("TampereBNB.pkl")
unpickled_df = pd.read_pickle("TampereBNB.pkl")

In [14]:
unpickled_df.head()

,Region,Price,Type,Size,Floor,Construction,Condition,Elevator,Longitude,Latitude
0,Tesoma,€180,Three rooms,74.0,7,1966,unknown,Yes,23.61033840613356,61.51083191092233
1,Takahuhti,€243,Two rooms,57.5,4,1972,good,Yes,23.860745947641902,61.49763715174763
2,Hervanta,€177,Studio apartment,27.0,0,2019,good,Yes,23.84882877070141,61.45446021008072
3,Niemenranta,€153,Studio apartment,24.5,1,2017,good,No,23.690050674048194,61.51717521456669
4,Tammela,€363,Three rooms,73.0,4,1971,good,Yes,23.77786391016316,61.50136263215722


In [15]:
#gettign a summary of the data set
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Region        1080 non-null   object
 1   Price         1080 non-null   object
 2   Type          1080 non-null   object
 3   Size          1080 non-null   object
 4   Floor         1080 non-null   object
 5   Construction  1080 non-null   int64 
 6   Condition     1080 non-null   object
 7   Elevator      1080 non-null   object
 8   Longitude     1080 non-null   object
 9   Latitude      1080 non-null   object
dtypes: int64(1), object(9)
memory usage: 84.5+ KB


In [16]:
#change data format
unpickled_df["Floor"] = pd.to_numeric(unpickled_df["Floor"])
unpickled_df["Size"] = pd.to_numeric(unpickled_df["Size"])
unpickled_df["Longitude"] = pd.to_numeric(unpickled_df["Longitude"])
unpickled_df["Latitude"] = pd.to_numeric(unpickled_df["Latitude"])

In [17]:
#check changes
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Region        1080 non-null   object 
 1   Price         1080 non-null   object 
 2   Type          1080 non-null   object 
 3   Size          1080 non-null   float64
 4   Floor         1080 non-null   int64  
 5   Construction  1080 non-null   int64  
 6   Condition     1080 non-null   object 
 7   Elevator      1080 non-null   object 
 8   Longitude     1080 non-null   float64
 9   Latitude      1080 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 84.5+ KB


In [18]:
#save and load latest changes
unpickled_df.to_pickle("TampereBNB.pkl")
unpickled_df = pd.read_pickle("TampereBNB.pkl")
unpickled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Region        1080 non-null   object 
 1   Price         1080 non-null   object 
 2   Type          1080 non-null   object 
 3   Size          1080 non-null   float64
 4   Floor         1080 non-null   int64  
 5   Construction  1080 non-null   int64  
 6   Condition     1080 non-null   object 
 7   Elevator      1080 non-null   object 
 8   Longitude     1080 non-null   float64
 9   Latitude      1080 non-null   float64
dtypes: float64(3), int64(2), object(5)
memory usage: 84.5+ KB
